In [1]:
import requests 
from bs4 import BeautifulSoup

In [2]:
def extractIntFromText(text):
    return int(text.replace(u'\xa0', u''))

In [3]:
def extractGenericLike(soup,classname):
    like_button = soup.findAll("button", { "class" : classname })[0]
    like_count_str = like_button.findAll("span", { "class" : "yt-uix-button-content" })[0].text
    like_count = extractIntFromText(like_count_str)
    return like_count

In [4]:
def getSoupFromUrl(url):
    #Execute q request toward Youtube
    request = requests.get(url)
    #parse the restult of the request
    soup = BeautifulSoup(request.text, 'html.parser')
    return soup

In [5]:
def extractMetricsFromUrl(url):
    
    soup = getSoupFromUrl(url)
    #GEt the view count
    view_count_str = soup.findAll("div", { "class" : "watch-view-count" })[0].text
    view_count = extractIntFromText(view_count_str)

    # Get the like count
    like_count = extractGenericLike(soup,"like-button-renderer-like-button")
    # Get the like count
    dislike_count = extractGenericLike(soup,"like-button-renderer-dislike-button")

    metrics = {}
    metrics['title'] = soup.title.text
    #metrics['view_count'] = view_count
    #metrics['like_count'] = like_count
    #metrics['dislike_count'] = dislike_count
    metrics['indicator'] = 1000.* (like_count - dislike_count) / view_count

    print '======================================================================================='
    print '==  Handling ' , soup.title.text
    print '==  The like count is', like_count, ' and dislike ', dislike_count
    print '==  Popularity indicator is ' , metrics['indicator']
    print '======================================================================================='
    return metrics

    #extractMetricsFromUrl('https://www.youtube.com/watch?v=B3eAMGXFw1o')
    #extractMetricsFromUrl('https://www.youtube.com/watch?v=Ao8cGLIMtvg')
    #extractMetricsFromUrl('https://www.youtube.com/watch?v=lWA2pjMjpBs')

In [6]:
def getAllMetricsForArtist(artist,MAX_PAGE):
    all_metrics = []
    for i in range(0,MAX_PAGE):
        url = 'https://www.youtube.com/results?search_query='+artist+'&page='+str(i)
        soup = getSoupFromUrl(url)
        tile_links = soup.findAll("a", { "class" : "yt-uix-tile-link" })
        for tile_link in tile_links:
            if 'watch' in tile_link['href']:
                url = 'https://www.youtube.com' + tile_link['href']
                metrics_for_url = extractMetricsFromUrl(url)
                all_metrics.append(metrics_for_url)
    return all_metrics

In [7]:
def getPopularityForArtist(artist,maxpage):
    return max(getAllMetricsForArtist(artist,maxpage))

In [8]:
rihanna = getAllMetricsForArtist('rihanna',2)
beyonce = getAllMetricsForArtist('beyonce',2)

==  Handling  Rihanna - Bitch Better Have My Money (Explicit) - YouTube
==  The like count is 1077539  and dislike  150849
==  Popularity indicator is  14.9393971744
==  Handling  Rihanna - American Oxygen - YouTube
==  The like count is 457866  and dislike  31094
==  Popularity indicator is  7.44467080543
==  Handling  Rihanna - Diamonds - YouTube
==  The like count is 2154187  and dislike  130908
==  Popularity indicator is  3.03900706767
==  Handling  Rihanna And Kanye West And Paul McCartney - FourFiveSeconds - YouTube
==  The like count is 1373720  and dislike  96652
==  Popularity indicator is  4.11656793039
==  Handling  Rihanna - Man Down - YouTube
==  The like count is 949242  and dislike  58877
==  Popularity indicator is  2.42746322398
==  Handling  Rihanna - Pour It Up (Explicit) - YouTube
==  The like count is 682830  and dislike  237529
==  Popularity indicator is  2.37439560757
==  Handling  Rihanna - Stay ft. Mikky Ekko - YouTube
==  The like count is 1582040  and disli